# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [48]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore as score
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'LDL)NA TGNMS BACGOSAS AQ/LE(EERTK', 'calculationPrice': 'iexlasttrade', 'open': 0, 'openTime': None, 'openSource': 'faoificl', 'close': 0, 'closeTime': None, 'closeSource': 'ofiifcal', 'high': 0, 'highTime': None, 'highSource': None, 'low': 0, 'lowTime': None, 'lowSource': None, 'latestPrice': 146.187, 'latestSource': 'IEX Last Trade', 'latestTime': 'July 9, 2021', 'latestUpdate': 1629011157011, 'latestVolume': None, 'iexRealtimePrice': 151.198, 'iexRealtimeSize': 352, 'iexLastUpdated': 1706150350902, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 145.77, 'previousVolume': 105850971, 'change': 1.84, 'changePercent': 0.0132, 'volume': None, 'iexMarketPercent': 0.01386723467009371, 'iexVolume': 1385850, 'avgTotalVolume': 81879774, 'iexBidPric

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_strings = []

for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index=my_columns
        ),
            ignore_index=True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,155.100,51.78,N/A
1,AAL,21.410,-1.47,N/A
2,AAP,215.040,23.5,N/A
3,AAPL,147.942,32.96,N/A
4,ABBV,121.910,41.25,N/A
...,...,...,...,...
500,YUM,122.110,32.68,N/A
501,ZBH,164.160,59.05,N/A
502,ZBRA,572.360,47.62,N/A
503,ZION,53.410,6.53,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)

In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,693.050,5.17,N/A
1,CINF,121.820,6.32,N/A
2,ZION,53.410,6.53,N/A
3,AFL,55.760,6.91,N/A
4,UNM,28.920,7.52,N/A
5,HRB,24.350,8.08,N/A
6,CE,156.520,8.8,N/A
7,COO,428.850,8.83,N/A
8,KIM,21.510,9.19,N/A
9,CSX,33.221,9.33,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,693.050,5.17,28
1,CINF,121.820,6.32,164
2,ZION,53.410,6.53,374
3,AFL,55.760,6.91,358
4,UNM,28.920,7.52,691
5,HRB,24.350,8.08,821
6,CE,156.520,8.8,127
7,COO,428.850,8.83,46
8,KIM,21.510,9.19,929
9,CSX,33.221,9.33,602


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [26]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    ]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value / ebitda
            
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
            
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
                            pd.Series([
                                symbol,
                                data[symbol]['quote']['latestPrice'],
                                'N/A',
                                data[symbol]['quote']['peRatio'],
                                'N/A',
                                data[symbol]['advanced-stats']['priceToBook'],
                                'N/A',
                                data[symbol]['advanced-stats']['priceToSales'],
                                'N/A',
                                ev_to_ebitda,
                                'N/A',
                                ev_to_gross_profit,
                                'N/A',
                                'N/A'
                            ],
                            index = rv_columns),
                                ignore_index=True
                            )

In [27]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.360,N/A,51.27,N/A,9.41,N/A,7.86,N/A,31.687102,N/A,14.929073,N/A,N/A
1,AAL,21.480,N/A,-1.49,N/A,-1.68,N/A,1.03,N/A,-3.659697,N/A,3.021369,N/A,N/A
2,AAP,221.660,N/A,23,N/A,4,N/A,1.31,N/A,11.611285,N/A,2.792912,N/A,N/A
3,AAPL,151.984,N/A,33.81,N/A,35.29,N/A,7.52,N/A,24.285970,N/A,18.547056,N/A,N/A
4,ABBV,121.030,N/A,41.57,N/A,15.58,N/A,4.19,N/A,13.271708,N/A,8.922097,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.430,N/A,32.34,N/A,-4.54,N/A,6.23,N/A,23.614073,N/A,10.560707,N/A,N/A
501,ZBH,159.750,N/A,58.46,N/A,2.75,N/A,4.7,N/A,28.122351,N/A,7.945223,N/A,N/A
502,ZBRA,564.300,N/A,47.5,N/A,12.54,N/A,6.3,N/A,33.928398,N/A,13.487206,N/A,N/A
503,ZION,53.830,N/A,6.27,N/A,1.15,N/A,2.41,N/A,4.745503,N/A,2.310637,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [30]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,237.74,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.86,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.01,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,28.77,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.34,N/A,14.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.65,N/A,33.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.57,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2633.28,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.35,N/A,31.33,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
327,NBL,8.74,N/A,-0.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [40]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    
    rv_dataframe.loc[:, column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [42]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [86]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
    }

for metric in metrics.keys():
    
    for row in rv_dataframe.index:
        
        entire_column = rv_dataframe[metric]
        value_at_rowXcolumn = rv_dataframe.loc[row, metric]
        rv_dataframe.loc[row, metrics[metric]] = score(entire_column, value_at_rowXcolumn) / 100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.180,1774,7.48,0.38,0.5389,0.16,0.4154,0.16,3.370840,0.18,0.396311,0.06,5.86139
1,MCK,192.100,260,-6.75,0.24,-1472.1000,0.02,0.1278,0.08,7.074118,0.48,2.506519,0.6,6.13861
2,AIV,7.250,6896,-82.30,0.08,2.0000,0.9,-31.2600,0.02,-44.843328,0.02,-54.655751,0.02,6.65347
3,AIG,47.360,1055,-10.58,0.22,0.6320,0.2,0.8900,0.42,5.814253,0.34,0.889411,0.16,6.77228
4,L,56.100,891,-412.32,0.02,0.8390,0.26,1.0164,0.46,7.894347,0.7,0.986479,0.2,7.16832
5,HFC,31.970,1563,-35.23,0.12,0.9829,0.32,0.4530,0.2,7.883460,0.66,2.612160,0.64,8.51485
6,AAL,21.480,2327,-1.49,0.3,-1.6800,0.12,1.0300,0.48,-3.659697,0.08,3.021369,0.76,9.66337
7,ABC,118.780,420,-6.43,0.26,-87.0600,0.04,0.1225,0.06,8.633808,0.8,4.153580,0.92,10.0198
8,BA,243.300,205,-12.23,0.2,-7.8700,0.1,2.5200,0.96,-18.863442,0.04,-40.787369,0.04,10.2376
9,ALL,138.220,361,11.90,0.66,1.6700,0.73,0.8513,0.4,2.494354,0.12,0.826982,0.14,11.0297


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [87]:
from statistics import mean

for row in rv_dataframe.index:
    
    value_percentiles = []
    
    for metric in metrics.keys():
        
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.180,1774,7.48,0.38,0.5389,0.16,0.4154,0.16,3.370840,0.18,0.396311,0.06,0.188
1,MCK,192.100,260,-6.75,0.24,-1472.1000,0.02,0.1278,0.08,7.074118,0.48,2.506519,0.6,0.284
2,AIV,7.250,6896,-82.30,0.08,2.0000,0.9,-31.2600,0.02,-44.843328,0.02,-54.655751,0.02,0.208
3,AIG,47.360,1055,-10.58,0.22,0.6320,0.2,0.8900,0.42,5.814253,0.34,0.889411,0.16,0.268
4,L,56.100,891,-412.32,0.02,0.8390,0.26,1.0164,0.46,7.894347,0.7,0.986479,0.2,0.328
5,HFC,31.970,1563,-35.23,0.12,0.9829,0.32,0.4530,0.2,7.883460,0.66,2.612160,0.64,0.388
6,AAL,21.480,2327,-1.49,0.3,-1.6800,0.12,1.0300,0.48,-3.659697,0.08,3.021369,0.76,0.348
7,ABC,118.780,420,-6.43,0.26,-87.0600,0.04,0.1225,0.06,8.633808,0.8,4.153580,0.92,0.416
8,BA,243.300,205,-12.23,0.2,-7.8700,0.1,2.5200,0.96,-18.863442,0.04,-40.787369,0.04,0.268
9,ALL,138.220,361,11.90,0.66,1.6700,0.73,0.8513,0.4,2.494354,0.12,0.826982,0.14,0.41


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [88]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.180,1774,7.48,0.38,0.5389,0.16,0.4154,0.16,3.370840,0.18,0.396311,0.06,0.188
1,AIV,7.250,6896,-82.30,0.08,2.0000,0.9,-31.2600,0.02,-44.843328,0.02,-54.655751,0.02,0.208
2,AIG,47.360,1055,-10.58,0.22,0.6320,0.2,0.8900,0.42,5.814253,0.34,0.889411,0.16,0.268
3,BA,243.300,205,-12.23,0.2,-7.8700,0.1,2.5200,0.96,-18.863442,0.04,-40.787369,0.04,0.268
4,MCK,192.100,260,-6.75,0.24,-1472.1000,0.02,0.1278,0.08,7.074118,0.48,2.506519,0.6,0.284
5,L,56.100,891,-412.32,0.02,0.8390,0.26,1.0164,0.46,7.894347,0.7,0.986479,0.2,0.328
6,STX,91.560,546,23.08,1,0.0000,0.14,0.0000,0.04,3.001258,0.14,1.455747,0.34,0.332
7,AAL,21.480,2327,-1.49,0.3,-1.6800,0.12,1.0300,0.48,-3.659697,0.08,3.021369,0.76,0.348
8,HFC,31.970,1563,-35.23,0.12,0.9829,0.32,0.4530,0.2,7.883460,0.66,2.612160,0.64,0.388
9,ALL,138.220,361,11.90,0.66,1.6700,0.73,0.8513,0.4,2.494354,0.12,0.826982,0.14,0.41


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [89]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [90]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.180,1774,7.48,0.38,0.5389,0.16,0.4154,0.16,3.370840,0.18,0.396311,0.06,0.188
1,AIV,7.250,6896,-82.30,0.08,2.0000,0.9,-31.2600,0.02,-44.843328,0.02,-54.655751,0.02,0.208
2,AIG,47.360,1055,-10.58,0.22,0.6320,0.2,0.8900,0.42,5.814253,0.34,0.889411,0.16,0.268
3,BA,243.300,205,-12.23,0.2,-7.8700,0.1,2.5200,0.96,-18.863442,0.04,-40.787369,0.04,0.268
4,MCK,192.100,260,-6.75,0.24,-1472.1000,0.02,0.1278,0.08,7.074118,0.48,2.506519,0.6,0.284
5,L,56.100,891,-412.32,0.02,0.8390,0.26,1.0164,0.46,7.894347,0.7,0.986479,0.2,0.328
6,STX,91.560,546,23.08,1,0.0000,0.14,0.0000,0.04,3.001258,0.14,1.455747,0.34,0.332
7,AAL,21.480,2327,-1.49,0.3,-1.6800,0.12,1.0300,0.48,-3.659697,0.08,3.021369,0.76,0.348
8,HFC,31.970,1563,-35.23,0.12,0.9829,0.32,0.4530,0.2,7.883460,0.66,2.612160,0.64,0.388
9,ALL,138.220,361,11.90,0.66,1.6700,0.73,0.8513,0.4,2.494354,0.12,0.826982,0.14,0.41


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')

rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
    }

for column in column_formats.keys():
    
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [99]:
writer.save()